In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as bs
from selenium.webdriver import ActionChains as ac
import pandas as pd
import time
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
import re

options = Options()

In [2]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
options.add_argument('user-agent=' + user_agent)
options.add_experimental_option('detach', True)
options.add_argument('--incognito')

In [3]:
# 지호 크롬드라이버
path = 'C:\전주시공모전\chromedriver_win32/chromedriver.exe'
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver = webdriver.Chrome(executable_path = path, options = options)

In [5]:
#호텔 링크 가져오기
hotel_links = pd.read_csv('hotel_links.csv', encoding = 'cp949')
hotel_links.head()

only_link = hotel_links["href"].values.tolist()

In [ ]:
only_link

In [178]:
# 데이터프레임으로 만들 딕셔너리 정의
reviews = {'score':[], 'text':[],'date':[],'title':[]}

In [70]:
hotel_links

,Unnamed: 0,호텔이름,href
0,0,전주 산정동 코자자,https://www.agoda.com/ko-kr/jeonju-sanjeong-do...
1,1,전주 금암동 뷰티,https://www.agoda.com/ko-kr/jeonju-geumam-dong...
2,2,전주 금암동 스카이쇼1 무인텔,https://www.agoda.com/ko-kr/jeonju-geumam-dong...
3,3,전주 금암동 타임,https://www.agoda.com/ko-kr/jeonju-geumam-dong...
4,4,전주 중화산동 조아,https://www.agoda.com/ko-kr/jeonju-junghwasan-...
5,5,전주 산정동 호텔 샾(Hotel Sharp),https://www.agoda.com/ko-kr/jeonju-sanjeong-do...
6,6,전주 금암동 블루원,https://www.agoda.com/ko-kr/jeonju-geumam-dong...
7,7,전주 덕진동 핀앤핏하우스,https://www.agoda.com/ko-kr/jeonju-deokjin-don...
8,8,전주 우아동 허브,https://www.agoda.com/ko-kr/eonju-ua-dong-herb...
9,9,전주 덕진동 HOTEL THE H(더 에이치),https://www.agoda.com/ko-kr/jeonju-deokjin-don...


In [179]:
for i in range(49,50):
    driver.get(only_link[i])
    time.sleep(2)
    # 리뷰 영역까지 스크롤
    scroll_point = driver.find_element_by_id('reviewFilterSection')
    ac(driver).move_to_element(scroll_point).perform()

    html = driver.page_source
    soup = bs(html, 'html.parser')

    driver.implicitly_wait(10)

    # 다시 파싱
    html = driver.page_source
    soup = bs(html, 'html.parser')

    #총 리뷰 개수 찾기 : for문을 몇 번 돌릴지 구하기위함 (버튼을 몇 번 눌러야 할지) 
    html = driver.page_source
    soup = bs(html, 'html.parser')

    #실제 이용후기 개수
    try:
        review_sum = soup.select('span.Review__SummaryContainer--left.Review__SummaryContainer__Text')[0].text
        if "실제 이용후기" in review_sum:
            try:
                review_sum = review_sum.split(" ")[3][:-1]
            except IndexError:
                review_sum = review_sum.split(" ")[1][:-1]
        if review_sum.startswith("이용후기"):
            review_sum = review_sum.split(" ")[1][:-1]
    except IndexError:
        pass   

    review_sum = review_sum.replace(",", "")

    try:
        pages = int(review_sum)//20 + 1
        for k in range(1,pages+1):
            html = driver.page_source
            soup = bs(html, 'html.parser')

        # 리뷰 영역까지 스크롤
        try:
            scroll_point = driver.find_element_by_id('review-0')
            ac(driver).move_to_element(scroll_point).perform()
                    # 다시 파싱
            html = driver.page_source
            soup = bs(html, 'html.parser')

            for k in range(0,9): #리뷰 개수에 따라 range 달라짐
                date_ = soup.select(f'#review-{k} > div.Review-comment-right > div.Review-comment-bubble > div.Review-statusBar > div > div > span')
                try:
                    year = date_[0].text.split(" ")[1][:-1]
                    month = date_[0].text.split("년 ")[1].split("월")[0].zfill(2)
                    day = date_[0].text.split("월 ")[1].split("일")[0].zfill(2)
                    yyyymmdd = f"{year}{month}{day}"
                    if int(year) >= 2018:
                        score = soup.select(f'#review-{k} > div.Review-comment-left > div > div.Review-comment-leftHeader > div.Review-comment-leftScore')
                        text = soup.select(f'#review-{k} > div.Review-comment-right > div.Review-comment-bubble > div.Review-comment-body > p.Review-comment-bodyText')
                        reviews['score'].append(score[0].text)
                        reviews['text'].append(text[0].text)
                        reviews['date'].append(yyyymmdd)
                        hotel = soup.select('.HeaderCerebrum__AdaName')
                        reviews['title'].append(hotel[0].text)
                    else:
                        break  
                except IndexError:
                    pass
        except NoSuchElementException:
            pass
    except ValueError:
        pass

    #버튼누르기       
    try:
        button = driver.find_element_by_xpath('//*[@id="reviewSection"]/div[3]/div/span[3]/i')
        ac(driver).move_to_element(button).click().perform()
        time.sleep(2)
    except NoSuchElementException:
        pass
    
    time.sleep(2)

In [164]:
#얘가 리뷰 엄청 많은거
# 전주 우아동 첫마중 같이 2페이지까지밖에 수집 안됨..왜 그럴까
for i in range(41,42):
    driver.get(only_link[i])
    time.sleep(2)
    # 리뷰 영역까지 스크롤
    scroll_point = driver.find_element_by_id('reviewFilterSection')
    ac(driver).move_to_element(scroll_point).perform()

    html = driver.page_source
    soup = bs(html, 'html.parser')

    driver.implicitly_wait(10)

    # 다시 파싱
    html = driver.page_source
    soup = bs(html, 'html.parser')

    #총 리뷰 개수 찾기 : for문을 몇 번 돌릴지 구하기위함 (버튼을 몇 번 눌러야 할지) 
    html = driver.page_source
    soup = bs(html, 'html.parser')

    #실제 이용후기 개수
    review_sum = 1368

    pages = int(review_sum)//20 + 1
    for k in range(1,pages+1):
        html = driver.page_source
        soup = bs(html, 'html.parser')

        # 리뷰 영역까지 스크롤
        scroll_point = driver.find_element_by_id('review-0')
        ac(driver).move_to_element(scroll_point).perform()
                # 다시 파싱
        html = driver.page_source
        soup = bs(html, 'html.parser')

        for k in range(0,9): #리뷰 개수에 따라 range 달라짐
            date_ = soup.select(f'#review-{k} > div.Review-comment-right > div.Review-comment-bubble > div.Review-statusBar > div > div > span')
            try:
                year = date_[0].text.split(" ")[1][:-1]
                month = date_[0].text.split("년 ")[1].split("월")[0].zfill(2)
                day = date_[0].text.split("월 ")[1].split("일")[0].zfill(2)
                yyyymmdd = f"{year}{month}{day}"
                if int(year) >= 2018:
                    score = soup.select(f'#review-{k} > div.Review-comment-left > div > div.Review-comment-leftHeader > div.Review-comment-leftScore')
                    text = soup.select(f'#review-{k} > div.Review-comment-right > div.Review-comment-bubble > div.Review-comment-body > p.Review-comment-bodyText')
                    reviews['score'].append(score[0].text)
                    reviews['text'].append(text[0].text)
                    reviews['date'].append(yyyymmdd)
                    hotel = soup.select('.HeaderCerebrum__AdaName')
                    reviews['title'].append(hotel[0].text)
                else:
                    break  
            except IndexError:
                pass

    #버튼누르기       
    button = driver.find_element_by_xpath('//*[@id="reviewSection"]/div[3]/div/span[3]/i')
    ac(driver).move_to_element(button).click().perform()
    time.sleep(2)
    
    time.sleep(2)

In [180]:
print(reviews)

{'score': ['8.4', '10.0'], 'text': ['주인 부부 내외가 친절하시고\n황금연휴 비싼 전주 숙박비 생각했을때 저렴한 편이었어요\n이용한지 오래되긴했습니다', '위치가 좋고 주차가 편하며 생수도 수건도 넉넉해 좋았습니다.  한옥마을을 둘러보고 조금 운전하여체크인하고 쉬다가 걸어나가서 한식을 먹고 가족이 치맥을 나누며 담소도 했습니다. 길 건너에 24시간 여는 짜글이집이 있어서 아침도 맛있게 먹었구요.'], 'date': ['20230513', '20220509'], 'title': ['전주 우아동 FX (Jeonju Ua-dong FX)', '전주 우아동 FX (Jeonju Ua-dong FX)']}


# 데이터프레임 정리

In [181]:
one = pd.DataFrame(data=reviews)
one = one.drop_duplicates()
one

,score,text,date,title
0,8.4,주인 부부 내외가 친절하시고\n황금연휴 비싼 전주 숙박비 생각했을때 저렴한 편이었어...,20230513,전주 우아동 FX (Jeonju Ua-dong FX)
1,10.0,위치가 좋고 주차가 편하며 생수도 수건도 넉넉해 좋았습니다. 한옥마을을 둘러보고 ...,20220509,전주 우아동 FX (Jeonju Ua-dong FX)


In [182]:
result1 = pd.concat([one,result1]).reset_index(drop=True)
result1 = result1.drop_duplicates().reset_index(drop=True)
result1

,score,text,date,title
0,8.4,주인 부부 내외가 친절하시고\n황금연휴 비싼 전주 숙박비 생각했을때 저렴한 편이었어...,20230513,전주 우아동 FX (Jeonju Ua-dong FX)
1,10.0,위치가 좋고 주차가 편하며 생수도 수건도 넉넉해 좋았습니다. 한옥마을을 둘러보고 ...,20220509,전주 우아동 FX (Jeonju Ua-dong FX)
2,6.0,깨끗하고 괜찮은데 개인적으로는 침대가 너무 푹신해서 잘 안맞음,20230420,전주 중화산동 라베호텔(LABE HOTEL) (Jeonju Junghwasandon...
3,10.0,잘 쉬고 갑니다\n직원분 너무 친절합니다\n베개가 한개라 안고잘 여분이 없어서 아쉬...,20230324,전주 중화산동 라베호텔(LABE HOTEL) (Jeonju Junghwasandon...
4,10.0,가족과 내집처럼펀하게 이용하고 왔습니다\n감사합니다,20230102,전주 중화산동 라베호텔(LABE HOTEL) (Jeonju Junghwasandon...
...,...,...,...,...
207,4.0,バスターミナルは近い。設備は古い。連泊したが掃除、タオル交換、ベッドメーキングはなく同じもの...,20230502,전주 금암동 타임
208,8.4,조용하고 편하게 잘 잤습니다!\n저희가 늦게 가서 주차자리가 없었는데 친절하게 주차...,20230531,전주 금암동 뷰티
209,8.4,화장실 청결상태가 매우 불량했고 담배 냄새 및 욕조에 머리카락 덩어리가 있어서 섬뜩...,20220201,전주 금암동 뷰티
210,8.0,風呂の栓がなく、しかも蛇口の取っ手が外されていて浴槽は使えなかったけど、それでもトータルとし...,20230505,전주 금암동 뷰티


In [183]:
result1.to_csv("agoda_review.csv", encoding = 'utf-8')

In [185]:
agoda_review = pd.read_csv("agoda_review.csv", encoding = 'utf-8')
agoda_review

,Unnamed: 0,score,text,date,title
0,0,8.4,주인 부부 내외가 친절하시고\n황금연휴 비싼 전주 숙박비 생각했을때 저렴한 편이었어...,20230513,전주 우아동 FX (Jeonju Ua-dong FX)
1,1,10.0,위치가 좋고 주차가 편하며 생수도 수건도 넉넉해 좋았습니다. 한옥마을을 둘러보고 ...,20220509,전주 우아동 FX (Jeonju Ua-dong FX)
2,2,6.0,깨끗하고 괜찮은데 개인적으로는 침대가 너무 푹신해서 잘 안맞음,20230420,전주 중화산동 라베호텔(LABE HOTEL) (Jeonju Junghwasandon...
3,3,10.0,잘 쉬고 갑니다\n직원분 너무 친절합니다\n베개가 한개라 안고잘 여분이 없어서 아쉬...,20230324,전주 중화산동 라베호텔(LABE HOTEL) (Jeonju Junghwasandon...
4,4,10.0,가족과 내집처럼펀하게 이용하고 왔습니다\n감사합니다,20230102,전주 중화산동 라베호텔(LABE HOTEL) (Jeonju Junghwasandon...
...,...,...,...,...,...
207,207,4.0,バスターミナルは近い。設備は古い。連泊したが掃除、タオル交換、ベッドメーキングはなく同じもの...,20230502,전주 금암동 타임
208,208,8.4,조용하고 편하게 잘 잤습니다!\n저희가 늦게 가서 주차자리가 없었는데 친절하게 주차...,20230531,전주 금암동 뷰티
209,209,8.4,화장실 청결상태가 매우 불량했고 담배 냄새 및 욕조에 머리카락 덩어리가 있어서 섬뜩...,20220201,전주 금암동 뷰티
210,210,8.0,風呂の栓がなく、しかも蛇口の取っ手が外されていて浴槽は使えなかったけど、それでもトータルとし...,20230505,전주 금암동 뷰티


# 언어 설정 부분 (제외함)

In [ ]:
# #아래로 된 삼각형 클릭
# button = driver.find_element_by_xpath('//*[@id="reviewFilterSection"]/div[1]/div[2]/span/i[2]')   
# # button = driver.find_element_by_xpath('//*[@id="reviewFilterSection"]/div[1]/div[3]/span/i[2]')
# ac(driver).move_to_element(button).click().perform()
# time.sleep(2)

# #한국어로 내려가기 및 클릭
# button = driver.find_element_by_xpath('//*[@id="reviews-language-filter_list"]/ul/li[2]')
# ac(driver).move_to_element(button).click().perform()
# time.sleep(2)

# #박스 접기
# button = driver.find_element_by_xpath('//*[@id="reviewFilterSection"]/div[1]/div[2]')
# ac(driver).move_to_element(button).click().perform()

# #한국어클릭
# time.sleep(2)
# button = driver.find_element_by_xpath('//*[@id="reviewFilterSection"]/div[1]/div[2]/div/ul/li[2]')

# ac(driver).move_to_element(button).click().perform
# time.sleep(2)